# Step 2: Generate Empirical Null Distributions

In [6]:
#import relevant libraries
import os, sys, json, math, random, pathlib, itertools, functools, warnings
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pickle
from nilearn.connectome import ConnectivityMeasure
from scipy.stats import spearmanr


# Set directory that has the fmri data
input_dir = 'C:/Users/mk7kc/Desktop/connectivity/vcap/vcap_Shen/dat'
os.chdir(input_dir)

In [ ]:
# Specify output dir (make if it doesn't already exist)
output_dir=''

if not os.path.isdir(output_dir):
    os.makedirs(output_dir)   

## Read in characterization data

In [ ]:
# Load in characterization data, remove participants with too high of a mean FD value (motion artifact too high)
# See exclusion.txt for more info
char_dir = 'C:/Users/mk7kc/Desktop/connectivity/vcap/vcap_MSDL/char_data/'
char_data = pd.read_csv('C:/Users/mk7kc/Desktop/connectivity/vcap/vcap_MSDL/char_data/char.csv')
exclude = [10, 43, 59, 78, 80, 83, 93]

# Demographic covariates
age = np.delete(char_data['Age'].values, exclude)
sex = np.delete(char_data['Sex'].values, exclude)

# Create function for faster extraction
def load_char(file, column):
    data = pd.read_csv(file)
    column_vals = data[column].values
    return np.delete(column_vals, exclude)

# Social Network metrics
SNComposite = load_char(
    os.path.join(char_dir, 'faOut1mlPromax_tenBergescores.csv'), 'ML1'
)

AnticipatedSupport = load_char(
    os.path.join(char_dir, 'socialnetwork_subscale_scores.csv'), 'PercSupp_Anticip'
)

SN_metrics = np.column_stack([SNComposite, AnticipatedSupport])


# Cognitive metrics
vocab = load_char(
    os.path.join(char_dir, 'vcap_cog_MK.csv'), 'vocab'
)

proc_speed = load_char(
    os.path.join(char_dir, 'vcap_cog_MK.csv'), 'proc_speed'
)

mem = load_char(
    os.path.join(char_dir, 'vcap_cog_MK.csv'), 'memory'
)

cog_metrics = np.column_stack([vocab, proc_speed, mem])  


variables={"Social Support":SN_metrics[:,0],
           "Anticipated Support":SN_metrics[:,1],
          "Vocab":cog_metrics[:,0],
          "Processing Speed":cog_metrics[:,1],
          "Memory":cog_metrics[:,2],
           "Age":age,
           "Sex":sex,
          }
df=pd.DataFrame(variables)
df.shape()
# Standardize values
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)

# Visualize pairplots and distributions
_ = sns.pairplot(df_scaled, vars=df.columns,kind="reg", diag_kind="kde",height=1.7)



### Choose fMRI Input

#### Option 1: Dictionary Learning output from high performance cluster processing of CPAC 4D standard to func outputs
Spatial maps generated via map learning algorithm based on spatial component sparsity then timeseries extracted for connectivty measures. To see model and gridsearch parameters see VCAP_CPAC folder for python scripts run on HPC for network & region extraction & connectome generation. **Note: participants that needed to be excluded were already excluded in the HPC step, so no need to do it here.**



In [ ]:
# Load in partial correlation data, use upper triangle data for features, Fishers Z transform 
fc_noGM_noLP = np.load('12comp_pcorr_noGM_noLP.npy')
fc_noGM_noLP = fc_noGM_noLP.reshape(92,71,71)
fc_noGM_noLP_feature=[]
for i in range(0,92):
    fv=fc_noGM_noLP[i][np.triu_indices(71, k = 1)] #k=1 excludes diagonal values
    fc_noGM_noLP_feature.append(fv)
x = np.arctanh(fc_noGM_noLP_feature) # 66 features

# Plot components using output from the best estimator model's components_img_
dictlearning_components_img = nib.load(input_dir+'.nii.gz')

# Plot atlas
# plot_prob_atlas(dictlearning_components_img,
#                 display_mode='z')
# plot_prob_atlas(dictlearning_components_img,
#                display_mode='x')
# plot_prob_atlas(dictlearning_components_img,
#                display_mode='y')

# Save images
#plot_prob_atlas(dictlearning_components_img,output_file=output_dir+'vcap_All_DictLearning_Components.png')
#plot_prob_atlas(dictlearning_components_img,
#                display_mode='x',output_file=output_dir+'vcap_All_DictLearning_Components_x.svg')

# Plot map for each component separately and save it into visualizations folder
# for i, cur_img in enumerate(iter_img(dictlearning_components_img)):
#     vis_name = os.path.join(output_dir,"Comp_y_"+str(i+1))
#     plot_stat_map(cur_img, display_mode="xz", title="Comp"+str(i+1),
#                   colorbar=True,output_file = vis_name) 
    
# Examine explained variance using best estimator model's scores   
scores = np.load(os.path.join(input_dir,'scores_12comp_220511_92participants_noGM_noLP.npy'))
print('My components explain %s perct. of the variance in the dataset' % str(round(np.sum(scores)*100,2)))

# Plot the explained variances per component
plt.figure(figsize=(10, 10))
numbers = np.arange(1,13)
plt.barh(numbers,scores)
plt.ylabel('Component #', size=30)
plt.xlabel('Explained Variance Ratio', size=30)
plt.yticks(np.arange(1,13))
plt.tick_params(axis='x', labelsize=20)
plt.tick_params(axis='y', labelsize=10)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
plt.tight_layout()
#plt.savefig(os.path.join(output_dir,"ExplainedVariance_80comp.svg"))

#### Option 2: Atlas output from fMRIPrep

In [33]:
# Specify atlas name in files
atlas_label = 'shen_atlas' # e.g. 'shen_atlas'
task_label = 'rest_1' # e.g. 'rest_1'
with open(f'vcap_{atlas_label}_ts_{task_label}.pkl', 'rb') as handle:
    fcdat = pickle.load(handle) 
    
# (Optional) Only extract nodes from certain networks, e.g. DMN network in Shen atlas
network_use = False
labels = pd.read_csv('shen_268_parcellation_networklabels.csv')
network_indices = labels.loc[labels['Network'] == 3].index # DMN = 3

# Extract time series from participants used for final analysis (remove those who meet exclusion criteria)
# List of participant IDs to exclude
exclude_id = char_data.iloc[exclude,0].values
exclude_id_str = {f"{n:03d}" for n in exclude_id}

# Get participant IDs
ids = sorted(fcdat.keys())
include_id = [include for include in ids if include not in exclude_id_str]
if network_use:
    ts = [fcdat[include][:, network_indices] for include in include_id]
else:
    ts = [fcdat[include] for include in include_id]             

# Connectivity
conn = ConnectivityMeasure(kind='partial correlation', vectorize=True, discard_diagonal=True)
x = np.arctanh(conn.fit_transform(ts))  # shape: (n_subjects, n_features)
print("X shape (subjects, features):", x.shape)

X shape (subjects, features): (92, 35778)


## Regress out covariates:
A linear regression analysis was performed at every feature to remove the effects of covariates. The residuals of this regression were then substituted for the feature values.

In [ ]:
covar = ['Age', 'Sex'] # add any other demo or covariates as needed
df_scaled_const = sm.add_constant(df_scaled[covar].values)

# # For loop version (takes longer)
# x_res = np.empty_like(x)
# for i in range(x.shape[1]):
#     x_res[:,i] = sm.OLS(x[:,i],df_scaled2).fit().resid 

# Vectorized version
M = np.eye(df_scaled_const.shape[0]) - df_scaled_const @ np.linalg.pinv(df_scaled_const)
x_res = M @ x    

## Define model

In [ ]:
# Specify where you saved your optimized parameters
input_dir = '' 
if input_dir:
    os.chdir(input_dir)

def read_opt(date_prefix,descriptor,model):
    opt_alpha = np.loadtxt(f"{date_prefix}_{descriptor}_alpha.csv", delimiter=",")
    opt_l1 = np.loadtxt(f"{date_prefix}_{descriptor}_l1_ratio.csv", delimiter=",") if model == 'elasticnet' else None
    # In case I only had one y target, cause then opt_alpha will be 1D array with shape (p,). 
    if opt_alpha.ndim == 1:
        opt_alpha = opt_alpha.reshape(-1, 1) # make sure it's 2D
    if (opt_l1 is not None) and opt_l1.ndim == 1:
        opt_l1 = opt_l1.reshape(-1, 1) # make sure it's 2D
    return opt_alpha,opt_l1

def run_Null(X, Y, target_names, model, perm=1000, test_size=0.2,random_seed=1,
           date_prefix=None, descriptor=None):
    """
    X: residualized connectivity data
    Y: predictor vars
    model: choose between ridge, lasso, elasticnet
    """
    model = model.lower()
    if model not in {'ridge','lasso','elasticnet'}:
        raise ValueError("check spelling of model type")
    n, n_targets = Y.shape
    
    # Load saved hyperparameters
    opt_alpha, opt_l1 = read_opt(date_prefix,descriptor,model)

    # Arrays to store results
    r2 = np.full((perm, n_targets), np.nan)
    mae = np.full((perm, n_targets), np.nan)
    rmse = np.full((perm, n_targets), np.nan)
    pearson = np.full((perm, n_targets), np.nan)
    spearman = np.full((perm, n_targets), np.nan)

    for p in range(perm):
        rs = np.random.RandomState(random_seed + p)
        idx = np.arange(n).copy()
        rs.shuffle(idx)
        n_test = int(np.ceil(n * test_size))
        test_idx, train_idx = idx[:n_test], idx[n_test:]
        X_train, X_test = X[train_idx], X[test_idx]

        for t in range(n_targets):
            y = Y[:, t]
            y_train, y_test = y[train_idx].copy(), y[test_idx] # make copy so that each shuffle is shuffle of original true y and not a shuffle of a shuffle (narrows effective space of randomizations and biases the null)
            rs.shuffle(y_train)
            
            # Build estimator
            if model == 'ridge':
                est = Ridge(alpha=opt_alpha[p,t])
                est_name = 'ridge'
            elif model == 'lasso':
                est = Lasso(alpha=opt_alpha[p,t],
                              random_state=random_seed + p, max_iter=1e7)
                est_name = 'lasso'
            elif model == 'elasticnet':
                est = ElasticNet(alpha=opt_alpha[p,t], 
                                   l1_ratio=opt_l1[p,t],
                                   random_state=random_seed + p, max_iter=1e7)
                est_name = 'elasticnet'
                
            pipe = Pipeline([
                ('sc', StandardScaler()),
                (est_name, est)
            ])

            pipe.fit(X_train, y_train)
            yhat = pipe.predict(X_test)

            # Metrics
            r2[p, t]   = r2_score(y_test, yhat)
            mae[p, t]  = mean_absolute_error(y_test, yhat)
            rmse[p, t] = mean_squared_error(y_test, yhat, squared=False)
            pearson[p, t]  = np.corrcoef(y_test, yhat)[1, 0] if y_test.std() > 0 else np.nan
            rho, _ = spearmanr(y_test, yhat)
            spearman[p, t] = rho if y_test.std() > 0 else np.nan
            

        if (p + 1) % max(1, perm // 5) == 0:
            print(f"{p+1}/{perm} permutations")

    if date_prefix and descriptor:
        # Save function
        def save(outcome, array):
            np.savetxt(os.path.join(output_dir, f'{date_prefix}_{descriptor}_{outcome}_NULL.csv'), array, delimiter=',')

        save('r2', r2)
        save('mae', mae)
        save('rmse', rmse)
        save('pearson', pearson)
        save('spearman', spearman)


    return {
        'r2': r2, 'mae': mae, 'rmse': rmse, 'pearson': pearson, 'spearman': spearman,
        'targets': target_names, 'model': model,
        'config': {'perm': perm, 'test_size': test_size}
    }

## Run model

In [ ]:
# Create final Y dataframe
Y = df_scaled.drop(columns=["Age", "Sex"]).values
y_metric_names = df_scaled.drop(columns=["Age", "Sex"]).columns.tolist()

# Run model
run_Null(x_res, Y, y_metric_names, model='elasticnet', perm=1000,
                    date_prefix='230301',descriptor='elasticnet_partialcorr_all')

## Visualize